In [1]:
# RNN Model (Individual + Multiple)

In [2]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics

from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils import np_utils

Using TensorFlow backend.


### Load Data
#### TODO: Specifiy Data Path Based on Participant/Your Computer

In [3]:
file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p3_data_FS4new.csv'
p3_data = pd.read_csv(file)
p3_data = p3_data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [4]:
file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p5_data_FS4new.csv'
p5_data = pd.read_csv(file)
p5_data = p5_data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [5]:
file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p7_data_FS4new.csv'
p7_data = pd.read_csv(file)
p7_data = p7_data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [6]:
file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p8_data_FS4new.csv'
p8_data = pd.read_csv(file)
p8_data = p8_data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [7]:
file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p9_data_FS4new.csv'
p9_data = pd.read_csv(file)
p9_data = p9_data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [8]:
file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4_new/p10_data_FS4new.csv'
p10_data = pd.read_csv(file)
p10_data = p10_data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

### Train - Test Split 

In [9]:
# Formulate Train Dataset


#train = p8_data.copy()
#train = train.append(p5_data)
#train = train.append(p3_data)
#train = train.append(p10_data)
#train = train.append(p9_data)

other = p5_data

bogus_test = other['engagement']
other_train, other_test, bogus1, bogus2 = train_test_split(other, bogus_test, test_size=0.5, shuffle=False)
train = other_train
test = other_test

### Create Windows

In [76]:
def create_windows(data):
    window_size = 50
    
    cols = []
    for i in range(0, window_size):
        cols.append(str(i))
    
    answer_X = pd.DataFrame(columns=cols)
    answer_y = pd.DataFrame(columns=cols)
    
    
    curr_session = -1
    window_start = -1
    
    for i,r in data.iterrows():
        this_row = r.name
        this_session = r['session_num']

        # we've encountered a new session
        if curr_session != this_session:
            # ignore remainder window
            curr_session = this_session
            window_start = this_row
            continue
        
        # reached window size
        if (this_row-window_start+1) == window_size:
            to_add = data.iloc[window_start:this_row+1]
            to_add_X = to_add.drop(columns=['timestamp', 'session_num', 'engagement'])
            to_add_y = to_add['engagement']

            new_X = {}
            new_y = {}
            for j in range(0, window_size):
                new_X[str(j)] = to_add_X.iloc[(j)]
                new_y[str(j)] = to_add_y.iloc[(j)]
                
            answer_X.append(new_X, ignore_index=True)
            answer_y.append(new_y, ignore_index=True)
            
            # reset variables
            curr_session = -1
            window_start = -1
            
    return answer_X, answer_y

In [78]:
X_train, y_train = create_windows(train)

In [82]:
X_train.shape

(0, 50)

### RNN Model

In [79]:
from keras.models import Sequential
from keras.layers import Embedding, Input
from keras.layers.merge import Concatenate
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Dropout, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.optimizers import RMSprop
from keras import metrics
from keras.models import Model

In [80]:
window_size = 50
input_dim = X_train[0].shape[1]
config = (window_size, input_dim)

model = Sequential()

model.add(LSTM(window_size, input_shape=config, return_sequences = True)) #return_sequences: if multiple LSTM layers

print(model.summary())
LOSS = 'binary_crossentropy' #classifying between 0 and 1
OPTIMIZER = 'rmsprop' #RMSprop typically works better for RNNs per Keras
model.compile(loss = LOSS, optimizer = OPTIMIZER, metrics = [metrics.binary_accuracy])

KeyError: 0

In [81]:
epochs = 5
batch_size = 16

model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size)

ValueError: Error when checking input: expected lstm_1_input to have 3 dimensions, but got array with shape (0, 50)

In [27]:
y_train

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Model Evaluation

In [ ]:
loss_and_metrics = nn_model.evaluate(X_test, y_test)
print(loss_and_metrics)

In [ ]:
scores = nn_model.predict_proba(X_test)
pred = nn_model.predict_classes(X_test)

In [ ]:
try:
    print("AUC:",roc_auc_score(y_test, scores)) 
except:
    print("AUC undefinied, only 1 class in test data")
    
print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred))